# Export the mean of the SD maps

In [1]:
# import the libraries
import ee
import pandas as pd
import os
import numpy as np
import random
from random import sample
import itertools 
import geopandas as gpd
from sklearn.metrics import r2_score
from termcolor import colored # this is allocate colour and fonts type for the print title and text
from IPython.display import display, HTML

In [2]:
# initialize the earth engine API
ee.Initialize()

In [3]:
# load the basic maps that needed for the analysis
# load the carbon concentration map
carbonConcentration = ee.Image("users/leonidmoore/ForestBiomass/Biome_level_Wood_Carbon_Conentration_Map")
# load the root shoot ratio map
rootShootRatio = ee.Image("users/leonidmoore/ForestBiomass/Root_shoot_ratio_Map").unmask()
# load the two composites tha will be used in the analysis
compositeImage =ee.Image("users/leonidmoore/ForestBiomass/20200915_Forest_Biomass_Predictors_Image")
compositeImageNew = ee.Image("projects/crowtherlab/Composite/CrowtherLab_Composite_30ArcSec")
# load the present forest cover
presentForestCover = compositeImage.select('PresentTreeCover')
potentialForestCover = ee.Image("users/leonidmoore/ForestBiomass/Bastin_et_al_2019_Potential_Forest_Cover_Adjusted").unmask().rename('PotentialForestCover')
potentialMask = potentialForestCover.gt(0)
# load the biome layer 
biomeLayer = compositeImage.select("WWF_Biome")
# define a pixel area layer with unit km2
pixelAreaMap = ee.Image.pixelArea().divide(10000);
# define the boundary geography reference
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False)

## load all the GS and SD maps of present biomass density

In [4]:
# load the GS carbon density maps of present biomass
carbonDensityGS_Max = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS1_MaxScaler_Present_density_Ensambled_Mean_20230427").unmask()
carbonDensityGS_Mean = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS1_MeanScaler_Present_density_Ensambled_Mean_20230427").unmask()
# load the SD carbon density maps of present biomass
carbonDensityWK =  ee.Image("users/leonidmoore/ForestBiomass/WalkerMap/reprojected_Walker_map_1km").unmask()
# since the SD map from Arnan is biomass density, therefore, we directly transfer thre biomass density into carbon density by multiplying the carbon concentration
carbonDensitySD =  ee.Image("users/leonidmoore/ForestBiomass/RemoteSensingModel/ESA_CCI_AGB_Map_bias_corrected_1km_2010").multiply(carbonConcentration).unmask()
carbonDensityHM =  ee.Image("users/leonidmoore/ForestBiomass/SpawnMap/Spawn_Harmonized_AGB_density_Map_1km").select('agb').unmask()

In [5]:
# compose the  images into a multiband image
carbonDensityImage = carbonDensityGS_Max.addBands(carbonDensityGS_Max).addBands(carbonDensityGS_Mean).addBands(carbonDensityWK).addBands(carbonDensitySD).addBands(carbonDensityHM).unmask()
# apply the reducer 
carbonDensityPresent = carbonDensityImage.reduce(ee.Reducer.mean()).mask(presentForestCover.gt(0))

## load all the GS and SD maps of potential biomass density

In [6]:
# load the potential carbon density for the GS models
potentialDensity_Max1 = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS1_MaxScaler_Potential_density_Ensambled_Mean_20230427").unmask()
potentialDensity_Max2 = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS2_MaxScaler_Potential_density_Ensambled_Mean_20230427").unmask()

potentialDensity_Mean1 = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS1_MeanScaler_Potential_density_Ensambled_Mean_20230427").unmask()
potentialDensity_Mean2 = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS2_MeanScaler_Potential_density_Ensambled_Mean_20230427").unmask()

# load the potential carbon density maps for SD models
potentialDensityWK1 = ee.Image("users/nordmannmoore/ForestBiomass/RemoteSensingModel/EnsambleMaps/Predicted_WK1_Potential_density_Ensambled_Mean").unmask()
potentialDensityWK2 = ee.Image("users/nordmannmoore/ForestBiomass/RemoteSensingModel/EnsambleMaps/Predicted_WK2_Potential_density_Ensambled_Mean").unmask()

potentialDensitySD1 = ee.Image("users/nordmannmoore/ForestBiomass/RemoteSensingModel/EnsambleMaps/Predicted_SD1_Potential_density_Ensambled_Mean").unmask()
potentialDensitySD2 = ee.Image("users/nordmannmoore/ForestBiomass/RemoteSensingModel/EnsambleMaps/Predicted_SD2_Potential_density_Ensambled_Mean").unmask()

potentialDensityHM1 = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_HM1_Potential_density_Ensambled_Mean").unmask()
potentialDensityHM2 = ee.Image("users/nordmannmoore/ForestBiomass/RemoteSensingModel/EnsambleMaps/Predicted_HM2_Potential_density_Ensambled_Mean").unmask()

## Adjust the map for each model

In [7]:
# define a functiion to do the values adjustment
def adjFunc(presentDen,potentialDen):
    # check the difference of the two density maps
    potentialHigher = potentialDen.subtract(presentDen).gte(0)
    potentialLower = potentialDen.subtract(presentDen).lt(0)
    # replace the lower potential value by present biomass density value
    adjPotentialDen = presentDen.multiply(potentialLower).add(potentialDen.multiply(potentialHigher))
    return(adjPotentialDen)

# GS max
tgbPotentialDensityGS_Max1 = adjFunc(presentDen = carbonDensityGS_Max,potentialDen = potentialDensity_Max1).multiply(rootShootRatio.add(1))
tgbPotentialDensityGS_Max2 = adjFunc(presentDen = carbonDensityGS_Max,potentialDen = potentialDensity_Max2).multiply(rootShootRatio.add(1))
tgbPresentDensityGS_Max = carbonDensityGS_Max.multiply(rootShootRatio.add(1))
# GS Mean 
tgbPotentialDensityGS_Mean1 = adjFunc(presentDen = carbonDensityGS_Mean,potentialDen = potentialDensity_Mean1).multiply(rootShootRatio.add(1))
tgbPotentialDensityGS_Mean2 = adjFunc(presentDen = carbonDensityGS_Mean,potentialDen = potentialDensity_Mean2).multiply(rootShootRatio.add(1))
tgbPresentDensityGS_Mean = carbonDensityGS_Mean.multiply(rootShootRatio.add(1))
# SD
tgbPotentialDensitySD1 = adjFunc(presentDen = carbonDensitySD,potentialDen = potentialDensitySD1).multiply(rootShootRatio.add(1))
tgbPotentialDensitySD2 = adjFunc(presentDen = carbonDensitySD,potentialDen = potentialDensitySD2).multiply(rootShootRatio.add(1))
tgbPresentDensitySD = carbonDensitySD.multiply(rootShootRatio.add(1))
# HM
tgbPotentialDensityHM1 = adjFunc(presentDen = carbonDensityHM,potentialDen = potentialDensityHM1).multiply(rootShootRatio.add(1))
tgbPotentialDensityHM2 = adjFunc(presentDen = carbonDensityHM,potentialDen = potentialDensityHM2).multiply(rootShootRatio.add(1))
tgbPresentDensityHM = carbonDensityHM.multiply(rootShootRatio.add(1))

# WK
tgbPotentialDensityWK1 = adjFunc(presentDen = carbonDensityWK,potentialDen = potentialDensityWK1).multiply(rootShootRatio.add(1))
tgbPotentialDensityWK2 = adjFunc(presentDen = carbonDensityWK,potentialDen = potentialDensityWK2).multiply(rootShootRatio.add(1))
tgbPresentDensityWK = carbonDensityWK.multiply(rootShootRatio.add(1))

## Export the full and net TGB potential maps

In [8]:
tgbPresentDensityImage = tgbPresentDensityGS_Max.addBands(tgbPresentDensityGS_Mean).addBands(tgbPresentDensitySD).addBands(tgbPresentDensityHM).addBands(tgbPresentDensityWK)
# apply the reducer 
tgbPresentDensity = tgbPresentDensityImage.reduce(ee.Reducer.mean())

# compose the  images into a multiband image
tgbPotentialDensityImage = tgbPotentialDensityGS_Max1.addBands(tgbPotentialDensityGS_Max2).addBands(tgbPotentialDensityGS_Mean1).addBands(tgbPotentialDensityGS_Mean2).addBands(tgbPotentialDensitySD1).addBands(tgbPotentialDensitySD2).addBands(tgbPotentialDensityHM1).addBands(tgbPotentialDensityHM2).addBands(tgbPotentialDensityWK1).addBands(tgbPotentialDensityWK2)
# apply the reducer 
tgbPotentialDensity = tgbPotentialDensityImage.reduce(ee.Reducer.mean())
tgbPotentialDensityNet = tgbPotentialDensity.subtract(tgbPresentDensity)

exportTask = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensity.mask(potentialMask),
                                                  description = 'Ensembled_full_potential_map_Export',
                                                  fileNamePrefix = 'ForestBiomassExport/Full_TGB_potential_Map_of_ensembled_mean',
                                                  region = unboundedGeo,
                                                  bucket = "crowtherlab_gcsb_lidong",
                                                  crs = 'EPSG:4326',
                                                  crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                  maxPixels = 1e13,
                                                  fileFormat = 'GeoTIFF')
# start the export task
exportTask.start()
# show the task status
exportTask.status()
exportTaskDiff = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensityNet.mask(potentialMask),
                                                      description = 'Ensembled_net_potential_map_Export',
                                                      fileNamePrefix = 'ForestBiomassExport/Net_TGB_potential_Map_of_ensembled_mean',
                                                      region = unboundedGeo,
                                                      bucket = "crowtherlab_gcsb_lidong",
                                                      crs = 'EPSG:4326',
                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                      maxPixels = 1e13,
                                                      fileFormat = 'GeoTIFF')
# start the export task
exportTaskDiff.start()
# show the task status
exportTaskDiff.status()


{'state': 'READY',
 'description': 'Ensembled_net_potential_map_Export',
 'creation_timestamp_ms': 1683295038627,
 'update_timestamp_ms': 1683295038627,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'WYPQEYTWTYR64SQQQVEE6WTL',
 'name': 'projects/earthengine-legacy/operations/WYPQEYTWTYR64SQQQVEE6WTL'}

## Export the full and net TGB potential maps for each model type

In [9]:
exportTaskGSMax1 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensityGS_Max1.mask(potentialMask),
                                                  description = 'GS_Max1_full_potential_map_Export',
                                                  fileNamePrefix = 'ForestBiomassExport/GS_Max1_Full_TGB_potential_Map',
                                                  region = unboundedGeo,
                                                  bucket = "crowtherlab_gcsb_lidong",
                                                  crs = 'EPSG:4326',
                                                  crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                  maxPixels = 1e13,
                                                  fileFormat = 'GeoTIFF')
# start the export task
exportTaskGSMax1.start()
# show the task status
exportTaskGSMax1.status()

exportTaskGSMax2 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensityGS_Max2.mask(potentialMask),
                                                  description = 'GS_Max2_full_potential_map_Export',
                                                  fileNamePrefix = 'ForestBiomassExport/GS_Max2_Full_TGB_potential_Map',
                                                  region = unboundedGeo,
                                                  bucket = "crowtherlab_gcsb_lidong",
                                                  crs = 'EPSG:4326',
                                                  crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                  maxPixels = 1e13,
                                                  fileFormat = 'GeoTIFF')
# start the export task
exportTaskGSMax2.start()
# show the task status
exportTaskGSMax2.status()



exportTaskGSMaxDiff1 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensityGS_Max1.subtract(tgbPresentDensityGS_Max).mask(potentialMask),
                                                      description = 'GS_Max1_net_potential_map_Export',
                                                      fileNamePrefix = 'ForestBiomassExport/GS_Max1_Net_TGB_potential_Map',
                                                      region = unboundedGeo,
                                                      bucket = "crowtherlab_gcsb_lidong",
                                                      crs = 'EPSG:4326',
                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                      maxPixels = 1e13,
                                                      fileFormat = 'GeoTIFF')
# start the export task
exportTaskGSMaxDiff1.start()
# show the task status
exportTaskGSMaxDiff1.status()


exportTaskGSMaxDiff2 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensityGS_Max2.subtract(tgbPresentDensityGS_Max).mask(potentialMask),
                                                      description = 'GS_Max2_net_potential_map_Export',
                                                      fileNamePrefix = 'ForestBiomassExport/GS_Max2_Net_TGB_potential_Map',
                                                      region = unboundedGeo,
                                                      bucket = "crowtherlab_gcsb_lidong",
                                                      crs = 'EPSG:4326',
                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                      maxPixels = 1e13,
                                                      fileFormat = 'GeoTIFF')
# start the export task
exportTaskGSMaxDiff2.start()
# show the task status
exportTaskGSMaxDiff2.status()


{'state': 'READY',
 'description': 'GS_Max2_net_potential_map_Export',
 'creation_timestamp_ms': 1683295052003,
 'update_timestamp_ms': 1683295052003,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'TLVPYFQ2JNQYEONUHPMGUC6P',
 'name': 'projects/earthengine-legacy/operations/TLVPYFQ2JNQYEONUHPMGUC6P'}

In [10]:
exportTaskGSMean1 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensityGS_Mean1.mask(potentialMask),
                                                  description = 'GS_Mean1_full_potential_map_Export',
                                                  fileNamePrefix = 'ForestBiomassExport/GS_Mean1_Full_TGB_potential_Map',
                                                  region = unboundedGeo,
                                                  bucket = "crowtherlab_gcsb_lidong",
                                                  crs = 'EPSG:4326',
                                                  crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                  maxPixels = 1e13,
                                                  fileFormat = 'GeoTIFF')
# start the export task
exportTaskGSMean1.start()
# show the task status
exportTaskGSMean1.status()

exportTaskGSMean2 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensityGS_Mean2.mask(potentialMask),
                                                  description = 'GS_Mean2_full_potential_map_Export',
                                                  fileNamePrefix = 'ForestBiomassExport/GS_Mean2_Full_TGB_potential_Map',
                                                  region = unboundedGeo,
                                                  bucket = "crowtherlab_gcsb_lidong",
                                                  crs = 'EPSG:4326',
                                                  crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                  maxPixels = 1e13,
                                                  fileFormat = 'GeoTIFF')
# start the export task
exportTaskGSMean2.start()
# show the task status
exportTaskGSMean2.status()



exportTaskGSMeanDiff1 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensityGS_Mean1.subtract(tgbPresentDensityGS_Mean).mask(potentialMask),
                                                      description = 'GS_Mean1_net_potential_map_Export',
                                                      fileNamePrefix = 'ForestBiomassExport/GS_Mean1_Net_TGB_potential_Map',
                                                      region = unboundedGeo,
                                                      bucket = "crowtherlab_gcsb_lidong",
                                                      crs = 'EPSG:4326',
                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                      maxPixels = 1e13,
                                                      fileFormat = 'GeoTIFF')
# start the export task
exportTaskGSMeanDiff1.start()
# show the task status
exportTaskGSMeanDiff1.status()


exportTaskGSMeanDiff2 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensityGS_Mean2.subtract(tgbPresentDensityGS_Mean).mask(potentialMask),
                                                      description = 'GS_Mean2_net_potential_map_Export',
                                                      fileNamePrefix = 'ForestBiomassExport/GS_Mean2_Net_TGB_potential_Map',
                                                      region = unboundedGeo,
                                                      bucket = "crowtherlab_gcsb_lidong",
                                                      crs = 'EPSG:4326',
                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                      maxPixels = 1e13,
                                                      fileFormat = 'GeoTIFF')
# start the export task
exportTaskGSMeanDiff2.start()
# show the task status
exportTaskGSMeanDiff2.status()

{'state': 'READY',
 'description': 'GS_Mean2_net_potential_map_Export',
 'creation_timestamp_ms': 1683295063892,
 'update_timestamp_ms': 1683295063892,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'USWLLDZZC24KVRCXGUB54SIH',
 'name': 'projects/earthengine-legacy/operations/USWLLDZZC24KVRCXGUB54SIH'}

In [11]:
exportTaskWK1 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensityWK1.mask(potentialMask),
                                                  description = 'WK1_full_potential_map_Export',
                                                  fileNamePrefix = 'ForestBiomassExport/WK1_Full_TGB_potential_Map',
                                                  region = unboundedGeo,
                                                  bucket = "crowtherlab_gcsb_lidong",
                                                  crs = 'EPSG:4326',
                                                  crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                  maxPixels = 1e13,
                                                  fileFormat = 'GeoTIFF')
# start the export task
exportTaskWK1.start()
# show the task status
exportTaskWK1.status()

exportTaskWK2 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensityWK2.mask(potentialMask),
                                                  description = 'WK2_full_potential_map_Export',
                                                  fileNamePrefix = 'ForestBiomassExport/WK2_Full_TGB_potential_Map',
                                                  region = unboundedGeo,
                                                  bucket = "crowtherlab_gcsb_lidong",
                                                  crs = 'EPSG:4326',
                                                  crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                  maxPixels = 1e13,
                                                  fileFormat = 'GeoTIFF')
# start the export task
exportTaskWK2.start()
# show the task status
exportTaskWK2.status()



exportTaskWKDiff1 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensityWK1.subtract(tgbPresentDensityWK).mask(potentialMask),
                                                      description = 'WK1_net_potential_map_Export',
                                                      fileNamePrefix = 'ForestBiomassExport/WK1_Net_TGB_potential_Map',
                                                      region = unboundedGeo,
                                                      bucket = "crowtherlab_gcsb_lidong",
                                                      crs = 'EPSG:4326',
                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                      maxPixels = 1e13,
                                                      fileFormat = 'GeoTIFF')
# start the export task
exportTaskWKDiff1.start()
# show the task status
exportTaskWKDiff1.status()


exportTaskWKDiff2 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensityWK2.subtract(tgbPresentDensityWK).mask(potentialMask),
                                                      description = 'WK2_net_potential_map_Export',
                                                      fileNamePrefix = 'ForestBiomassExport/WK2_Net_TGB_potential_Map',
                                                      region = unboundedGeo,
                                                      bucket = "crowtherlab_gcsb_lidong",
                                                      crs = 'EPSG:4326',
                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                      maxPixels = 1e13,
                                                      fileFormat = 'GeoTIFF')
# start the export task
exportTaskWKDiff2.start()
# show the task status
exportTaskWKDiff2.status()

{'state': 'READY',
 'description': 'WK2_net_potential_map_Export',
 'creation_timestamp_ms': 1683295066686,
 'update_timestamp_ms': 1683295066686,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'LJ5WZZK6NTXQAOHWKGXIKKWD',
 'name': 'projects/earthengine-legacy/operations/LJ5WZZK6NTXQAOHWKGXIKKWD'}

In [24]:
exportTaskHM1 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensityHM1.mask(potentialMask),
                                                  description = 'HM1_full_potential_map_Export',
                                                  fileNamePrefix = 'ForestBiomassExport/HM1_Full_TGB_potential_Map',
                                                  region = unboundedGeo,
                                                  bucket = "crowtherlab_gcsb_lidong",
                                                  crs = 'EPSG:4326',
                                                  crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                  maxPixels = 1e13,
                                                  fileFormat = 'GeoTIFF')
# start the export task
exportTaskHM1.start()
# show the task status
exportTaskHM1.status()

exportTaskHM2 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensityHM2.mask(potentialMask),
                                                  description = 'HM2_full_potential_map_Export',
                                                  fileNamePrefix = 'ForestBiomassExport/HM2_Full_TGB_potential_Map',
                                                  region = unboundedGeo,
                                                  bucket = "crowtherlab_gcsb_lidong",
                                                  crs = 'EPSG:4326',
                                                  crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                  maxPixels = 1e13,
                                                  fileFormat = 'GeoTIFF')
# start the export task
exportTaskHM2.start()
# show the task status
exportTaskHM2.status()



exportTaskHMDiff1 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensityHM1.subtract(tgbPresentDensityHM).mask(potentialMask),
                                                      description = 'HM1_net_potential_map_Export',
                                                      fileNamePrefix = 'ForestBiomassExport/HM1_Net_TGB_potential_Map',
                                                      region = unboundedGeo,
                                                      bucket = "crowtherlab_gcsb_lidong",
                                                      crs = 'EPSG:4326',
                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                      maxPixels = 1e13,
                                                      fileFormat = 'GeoTIFF')
# start the export task
exportTaskHMDiff1.start()
# show the task status
exportTaskHMDiff1.status()


exportTaskHMDiff2 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensityHM2.subtract(tgbPresentDensityHM).mask(potentialMask),
                                                      description = 'HM2_net_potential_map_Export',
                                                      fileNamePrefix = 'ForestBiomassExport/HM2_Net_TGB_potential_Map',
                                                      region = unboundedGeo,
                                                      bucket = "crowtherlab_gcsb_lidong",
                                                      crs = 'EPSG:4326',
                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                      maxPixels = 1e13,
                                                      fileFormat = 'GeoTIFF')
# start the export task
exportTaskHMDiff2.start()
# show the task status
exportTaskHMDiff2.status()

{'state': 'READY',
 'description': 'HM2_net_potential_map_Export',
 'creation_timestamp_ms': 1675161127047,
 'update_timestamp_ms': 1675161127047,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'U5NU4Y7VHI5B6G4R7IFMRYYO',
 'name': 'projects/earthengine-legacy/operations/U5NU4Y7VHI5B6G4R7IFMRYYO'}

In [25]:
exportTaskSD1 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensitySD1.mask(potentialMask),
                                                  description = 'SD1_full_potential_map_Export',
                                                  fileNamePrefix = 'ForestBiomassExport/SD1_Full_TGB_potential_Map',
                                                  region = unboundedGeo,
                                                  bucket = "crowtherlab_gcsb_lidong",
                                                  crs = 'EPSG:4326',
                                                  crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                  maxPixels = 1e13,
                                                  fileFormat = 'GeoTIFF')
# start the export task
exportTaskSD1.start()
# show the task status
exportTaskSD1.status()

exportTaskSD2 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensitySD2.mask(potentialMask),
                                                  description = 'SD2_full_potential_map_Export',
                                                  fileNamePrefix = 'ForestBiomassExport/SD2_Full_TGB_potential_Map',
                                                  region = unboundedGeo,
                                                  bucket = "crowtherlab_gcsb_lidong",
                                                  crs = 'EPSG:4326',
                                                  crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                  maxPixels = 1e13,
                                                  fileFormat = 'GeoTIFF')
# start the export task
exportTaskSD2.start()
# show the task status
exportTaskSD2.status()



exportTaskSDDiff1 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensitySD1.subtract(tgbPresentDensitySD).mask(potentialMask),
                                                      description = 'SD1_net_potential_map_Export',
                                                      fileNamePrefix = 'ForestBiomassExport/SD1_Net_TGB_potential_Map',
                                                      region = unboundedGeo,
                                                      bucket = "crowtherlab_gcsb_lidong",
                                                      crs = 'EPSG:4326',
                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                      maxPixels = 1e13,
                                                      fileFormat = 'GeoTIFF')
# start the export task
exportTaskSDDiff1.start()
# show the task status
exportTaskSDDiff1.status()


exportTaskSDDiff2 = ee.batch.Export.image.toCloudStorage(image = tgbPotentialDensitySD2.subtract(tgbPresentDensitySD).mask(potentialMask),
                                                      description = 'SD2_net_potential_map_Export',
                                                      fileNamePrefix = 'ForestBiomassExport/SD2_Net_TGB_potential_Map',
                                                      region = unboundedGeo,
                                                      bucket = "crowtherlab_gcsb_lidong",
                                                      crs = 'EPSG:4326',
                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                      maxPixels = 1e13,
                                                      fileFormat = 'GeoTIFF')
# start the export task
exportTaskSDDiff2.start()
# show the task status
exportTaskSDDiff2.status()

{'state': 'READY',
 'description': 'SD2_net_potential_map_Export',
 'creation_timestamp_ms': 1675161129989,
 'update_timestamp_ms': 1675161129989,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'PQA75UFIEH3U5XF4SYXCSVP3',
 'name': 'projects/earthengine-legacy/operations/PQA75UFIEH3U5XF4SYXCSVP3'}

## Generate the maps for Figure S6

In [9]:
# compose the  images into a multiband image
tgbPotentialDensityGS_Comp = tgbPotentialDensityGS_Max1.addBands(tgbPotentialDensityGS_Max2).addBands(tgbPotentialDensityGS_Mean1)
tgbPotentialDensityGS = tgbPotentialDensityGS_Comp.reduce(ee.Reducer.mean())

# compose the  images into a multiband image
tgbPotentialDensitySD_Comp = tgbPotentialDensitySD1.addBands(tgbPotentialDensitySD2).addBands(tgbPotentialDensityHM1).addBands(tgbPotentialDensityHM2).addBands(tgbPotentialDensityWK1).addBands(tgbPotentialDensityWK2)
# apply the reducer 
tgbPotentialDensitySD = tgbPotentialDensitySD_Comp.reduce(ee.Reducer.mean())


# get the difference layer of the fuul potential from both GS and SD
fullPotentialDensityDiff = tgbPotentialDensityGS.subtract(tgbPotentialDensitySD)

# get the present existing biomass for GS and SD
tgbPresentDensityGS_Comp = tgbPresentDensityGS_Max.addBands(tgbPresentDensityGS_Mean)
# apply the reducer 
tgbPresentDensityGS = tgbPresentDensityGS_Comp.reduce(ee.Reducer.mean())

tgbPresentDensitSD_Comp = tgbPresentDensitySD.addBands(tgbPresentDensityHM).addBands(tgbPresentDensityWK)
# apply the reducer 
tgbPresentDensitySD = tgbPresentDensitSD_Comp.reduce(ee.Reducer.mean())
# get the net potential for GS and SD
netPotentialDensityGS = tgbPotentialDensityGS.subtract(tgbPresentDensityGS)
netPotentialDensitySD = tgbPotentialDensitySD.subtract(tgbPresentDensitySD)
# create the net potential difference of this two types for GS and SD
netPotentialDensityDiff = netPotentialDensityGS.subtract(netPotentialDensitySD)
netPotentialDensityMean = netPotentialDensityGS.add(netPotentialDensitySD).divide(2)
# get the ratio of the net difference
netPotentialDensityRatio = netPotentialDensityDiff.divide(netPotentialDensityMean)

In [13]:
exportFullDiff01 = ee.batch.Export.image.toCloudStorage(image = fullPotentialDensityDiff.mask(potentialMask),
                                                      description = 'Full_potential_diff_GS_SD_map_Export',
                                                      fileNamePrefix = 'ForestBiomassExport/Full_TGB_potential_diff_Map_of_GS_SD',
                                                      region = unboundedGeo,
                                                      bucket = "crowtherlab_gcsb_lidong",
                                                      crs = 'EPSG:4326',
                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                      maxPixels = 1e13,
                                                      fileFormat = 'GeoTIFF')
# start the export task
exportFullDiff01.start()
# show the task status
exportFullDiff01.status()

exportNetDiff01 = ee.batch.Export.image.toCloudStorage(image = netPotentialDensityDiff.mask(potentialMask),
                                                      description = 'Net_potential_diff_GS_SD_map_Export',
                                                      fileNamePrefix = 'ForestBiomassExport/Net_TGB_potential_diff_Map_of_GS_SD',
                                                      region = unboundedGeo,
                                                      bucket = "crowtherlab_gcsb_lidong",
                                                      crs = 'EPSG:4326',
                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                      maxPixels = 1e13,
                                                      fileFormat = 'GeoTIFF')
# start the export task
exportNetDiff01.start()
# show the task status
exportNetDiff01.status()


exportNetRatio01 = ee.batch.Export.image.toCloudStorage(image = netPotentialDensityRatio.mask(potentialMask),
                                                      description = 'Net_potential_ratio_GS_SD_map_Export',
                                                      fileNamePrefix = 'ForestBiomassExport/Net_TGB_potential_ratio_Map_of_GS_SD',
                                                      region = unboundedGeo,
                                                      bucket = "crowtherlab_gcsb_lidong",
                                                      crs = 'EPSG:4326',
                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                      maxPixels = 1e13,
                                                      fileFormat = 'GeoTIFF')
# start the export task
exportNetRatio01.start()
# show the task status
exportNetRatio01.status()

{'state': 'READY',
 'description': 'Net_potential_ratio_GS_SD_map_Export',
 'creation_timestamp_ms': 1683295104449,
 'update_timestamp_ms': 1683295104449,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'PGIUHYGGDOCIWYUJA6OJ5FR6',
 'name': 'projects/earthengine-legacy/operations/PGIUHYGGDOCIWYUJA6OJ5FR6'}

## check the pixel number difference 

In [11]:
# calculate the pixels with GS is lower than the SD
# define the GS is larger map
lagerMap = netPotentialDensityDiff.mask(potentialMask).gt(0)
largerPixelNumber = lagerMap.reduceRegion(reducer= ee.Reducer.sum(),
                                          geometry= unboundedGeo,
                                          crs='EPSG:4326',
                                          crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                          maxPixels= 1e13)

smallerMap = netPotentialDensityDiff.mask(potentialMask).lt(0)
smallerPixelNumber = smallerMap.reduceRegion(reducer= ee.Reducer.sum(),
                                             geometry= unboundedGeo,
                                             crs='EPSG:4326',
                                             crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                             maxPixels= 1e13)

print(largerPixelNumber.getInfo())
print(smallerPixelNumber.getInfo())

{'mean': 33164412}
{'mean': 153139268}


In [12]:
33164412/(33164412+153139268)*100

17.801265117253724